In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize images to a common size
            images.append(img)
    return images

In [3]:
# Load and preprocess data
foul_images = load_images_from_folder('/kaggle/input/op-dataset/OP_Dataset/Train/foul')
non_foul_images = load_images_from_folder('/kaggle/input/op-dataset/OP_Dataset/Train/non_foul')

# Create labels (1 for foul, 0 for non-foul)
foul_labels = np.ones(len(foul_images))
non_foul_labels = np.zeros(len(non_foul_images))

# Combine data and labels
data = np.array(foul_images + non_foul_images)
labels = np.concatenate((foul_labels, non_foul_labels))

val_foul_images = load_images_from_folder("/kaggle/input/op-dataset/OP_Dataset/Valid/foul")
val_non_foul_images = load_images_from_folder("/kaggle/input/op-dataset/OP_Dataset/Valid/non_foul")

val_foul_labels = np.ones(len(val_foul_images))
val_non_foul_labels = np.zeros(len(val_non_foul_images))

val_data = np.array(val_foul_images + val_non_foul_images)
val_labels = np.concatenate((val_foul_labels, val_non_foul_labels))


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the ImageDataGenerator on your training data
datagen.fit(data)

# Load ResNet50 pre-trained model
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define a learning rate schedule
def schedule(epoch, lr):
    if epoch < 50:
        return 0.0001
    elif 50 <= epoch < 75:
        return 0.00001
    else:
        return 0.000001

# Use the learning rate scheduler during model training
lr_scheduler = LearningRateScheduler(schedule)

# Train the model
history = model.fit(datagen.flow(data, labels, batch_size=32),
                    epochs=100, validation_data=(val_data, val_labels),
                    callbacks=[lr_scheduler])

# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_data, val_labels)
print("Validation Accuracy:", val_accuracy)


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
38/38 [==============================] - 20s 371ms/step - loss: 0.8550 - accuracy: 0.5390 - val_loss: 0.7061 - val_accuracy: 0.5673 - lr: 1.0000e-04
Epoch 2/100
38/38 [==============================] - 13s 333ms/step - loss: 0.7392 - accuracy: 0.5714 - val_loss: 0.5652 - val_accuracy: 0.7285 - lr: 1.0000e-04
Epoch 3/100
38/38 [==============================] - 13s 331ms/step - loss: 0.6676 - accuracy: 0.6071 - val_loss: 0.5884 - val_accuracy: 0.6667 - lr: 1.0000e-04
Epoch 4/100
38/38 [==============================] - 13s 344ms/step - loss: 0.6205 - accuracy: 0.6404 - val_loss: 0.5308 - val_accuracy: 0.7439 - lr: 1.0000e-04
Epoch 5/100
38/38 [==============================] - 13s 330ms/step - loss: 0.6073 - accuracy: 0.6703 - val_loss: 0.5092 - val_accuracy: 0.7682 - lr: 1.0000e-04
Epoch 6/100
38/38 [==============================] - 13s 339ms/step - loss: 0.6167 - accuracy: 0.6545 - val_loss: 0.4979 - val_acc

In [6]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(val_data)
conf_matrix = confusion_matrix(val_labels, predictions > 0.5)
print("Confusion Matrix:")
print(conf_matrix)

15/15 [==============================] - 2s 50ms/step
Confusion Matrix:
[[235  14]
 [  4 200]]


In [7]:
model.save("foul_detection_model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [4]:
def detect_foul(image_path, foul_model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize image to match your model's input size
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = foul_model.predict(img)
    if prediction >= 0.5:
        return "foul"
    else:
        return "not-foul"

In [10]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Clean_Tackles/109.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: not-foul


In [11]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Fouls/10.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 1s 949ms/step
Detection Result: foul


In [12]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Fouls/108.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 1s 960ms/step
Detection Result: not-foul


In [13]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Clean_Tackles/111.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 1s 953ms/step
Detection Result: not-foul


In [17]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/working/foul_detection_model.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/football-tackles/var400/VAR/Fouls"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 23ms/step
Total fouls detected: 392


In [18]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/working/foul_detection_model.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/football-tackles/var400/VAR/Clean_Tackles"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 21ms/step
Total fouls detected: 18


In [ ]:
import wget
import os

def download_model(model_url, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    model_filename = os.path.join(save_path, 'foul_detection_model.h5')

    if not os.path.exists(model_filename):
        print(f"Downloading model from {model_url}")
        wget.download(model_url, out=model_filename)
        print("\nDownload complete.")
    else:
        print("Model already exists.")

    return model_filename

# Example usage:
model_url = 'https://example.com/path/to/foul_detection_model.h5'
save_path = '/path/to/save'

downloaded_model_path = download_model(model_url, save_path)

In [12]:
model_path = '/kaggle/input/resnet50/ResNet50.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/soccer-dataset/Soccer Dataset/Test/Foul/124.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 1s 956ms/step
Detection Result: not-foul


In [5]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/input/resnet50/ResNet50.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/videoframes/kaggle/working/image"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 25ms/step
Total fouls detected: 222
